In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import os
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import threading
import json

In [2]:
num_objects_json = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/objects')

In [3]:
num_objects = num_objects_json.json()
total_objects = num_objects['total']
object_IDs = num_objects['objectIDs']
object_IDs

[505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 515,
 516,
 517,
 519,
 520,
 521,
 522,
 523,
 524,
 525,
 526,
 527,
 528,
 529,
 530,
 531,
 532,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 540,
 541,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 550,
 551,
 552,
 553,
 554,
 555,
 556,
 559,
 560,
 561,
 562,
 564,
 565,
 566,
 567,
 568,
 569,
 570,
 571,
 572,
 573,
 574,
 575,
 576,
 577,
 578,
 579,
 580,
 581,
 582,
 583,
 584,
 585,
 586,
 587,
 588,
 589,
 590,
 591,
 592,
 593,
 594,
 595,
 596,
 597,
 598,
 599,
 600,
 601,
 602,
 603,
 604,
 605,
 606,
 607,
 608,
 609,
 610,
 611,
 612,
 613,
 614,
 615,
 616,
 617,
 618,
 619,
 620,
 621,
 622,
 623,
 624,
 625,
 626,
 627,
 628,
 630,
 631,
 632,
 633,
 634,
 635,
 636,
 637,
 638,
 639,
 640,
 641,
 642,
 643,
 644,
 645,
 646,
 647,
 648,
 649,
 650,
 651,
 652,
 653,
 654,
 655,
 656,
 657,
 658,
 659,
 660,
 661,
 662,
 663,
 664,
 665,
 666,
 667,
 668,
 669,
 670,
 671,
 672,
 673,
 674,
 675,
 676

In [4]:
df_info = pd.read_csv('image_information.csv')
print(df_info.shape[0])
df_info.head()

C:\Users\Jona\AppData\Local\Temp\ipykernel_11732\645308725.py:1: DtypeWarning: Columns (4,13,14,15,16,17,19,25,26,37,38,39,40,41,42,43,44,45,46,47,48,49,55,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df_info = pd.read_csv('image_information.csv')


493864


,Unnamed: 0,objectID,isHighlight,accessionNumber,accessionYear,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,...,classification,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,objectWikidata_URL,isTimelineWork,GalleryNumber
0,0,1,False,1979.486.1,1979.0,False,NaN,NaN,[],"[{'constituentID': 164292, 'role': 'Maker', 'n...",...,NaN,NaN,NaN,2024-08-14T04:52:51.407Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/1,NaN,NaN,False,NaN
1,1,2,False,1980.264.5,1980.0,False,NaN,NaN,[],"[{'constituentID': 1079, 'role': 'Maker', 'nam...",...,NaN,NaN,NaN,2024-08-14T04:52:51.407Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/2,NaN,NaN,False,NaN
2,2,3,False,67.265.9,1967.0,False,NaN,NaN,[],NaN,...,NaN,NaN,NaN,2021-04-06T04:41:04.967Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/3,NaN,NaN,False,NaN
3,3,4,False,67.265.10,1967.0,False,NaN,NaN,[],NaN,...,NaN,NaN,NaN,2024-01-10T04:57:19.843Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/4,NaN,NaN,False,NaN
4,4,5,False,67.265.11,1967.0,False,NaN,NaN,[],NaN,...,NaN,NaN,NaN,2024-01-10T04:57:19.843Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/search/5,NaN,NaN,False,NaN


In [10]:
url = 'https://www.metmuseum.org/art/collection/search/'

error_count = 0
last_error_print_time = 0
error_lock = threading.Lock()

def requests_met_api(i):
    global error_count, last_error_print_time
    while True:
        try:
            response = requests.get(url + str(i))
            soup = BeautifulSoup(response.text, "html.parser")
            img_tag = soup.find("img", id="artwork__image")
            if img_tag and "src" in img_tag.attrs:
                return [i, img_tag["src"]]
            else:
                return None
        except Exception:
            with error_lock:
                current_time = time.time()
                if current_time - last_error_print_time >= 60:
                    print('('+str(error_count)+')', 'Error, retrying in 1 minutes')
                    error_count += 1
                    last_error_print_time = current_time
            time.sleep(60)

start_time = time.perf_counter()
count_time = start_time

# add all dictionary responses to a list, will be converted to a DataFrame later
img_url_dict = {}

# run multiple requests at once
with ThreadPoolExecutor(max_workers=100) as executor:
    for count, result in enumerate(executor.map(requests_met_api, object_IDs), start=1):

        # print counter every 1000 reponses and how long it took
        if count % 1000 == 0:
            print('count:', str(count), '  time:', str(time.perf_counter() - count_time), '  total time', str(time.perf_counter() - start_time))
            count_time = time.perf_counter()
        
        if result != None:
            img_url_dict[result[0]] = result[1]

# convert the list of dicionatry responses to a DataFrame
with open('img_url.json', 'w') as jsonfile:
    json.dump(img_url_dict, jsonfile)

count: 1000   time: 37.9833881000086   total time 37.98339200000919
count: 2000   time: 32.043332499990356   total time 70.03030939999735
count: 3000   time: 32.22814289999951   total time 102.26447770000959
count: 4000   time: 28.387333099992247   total time 130.65189829999872
count: 5000   time: 31.53729900000326   total time 162.18929999999818
count: 6000   time: 29.80632789999072   total time 191.99721549999958
count: 7000   time: 29.765597100005834   total time 221.86765680000826
count: 8000   time: 32.11671309999656   total time 254.0571437000035
count: 9000   time: 31.250025799992727   total time 285.3845188999985
count: 10000   time: 33.09575030000997   total time 318.4803687000094
count: 11000   time: 29.705291799997212   total time 348.18583970000327
count: 12000   time: 32.30986480000138   total time 380.5738285000116
count: 13000   time: 31.779045100003714   total time 412.3927377000073
count: 14000   time: 28.983906299996306   total time 441.3767385000101
count: 15000   ti

In [11]:
with open('img_url.json', 'r') as f:
    URL_dict = json.load(f)

print(len(URL_dict))

348763
